In [8]:
import numpy as np
import astropy.io.fits as fits
import os
import sys
import glob
from datetime import datetime

In [9]:
def get_dark(expt):
    for i in glob.glob("./ngc6720/ngc6720/darks/*.fits"):
        try:
            img=fits.open(i)
        except:
            print(f"{i} cannot be open as a fits file")
            continue
        if img[0].header["EXPTIME"] == expt:
            return img
    
def get_flat(filter):
    for i in glob.glob("./ngc6720/ngc6720/flats/*.fits"):
        try:
           img=fits.open(i)
        except:
           print(f"{i} cannot be open as a fits file")
           continue
        if img[0].header["FILTER"] == filter:
            return img

In [10]:
for i in glob.glob("./ngc6720/ngc6720/*.fits"):
    try:
        im=fits.open(i)
    except:
        print(f"{i} cannot be open as a fits file")
        continue
    
    df = get_dark(im[0].header["EXPTIME"])
    ff = get_flat(im[0].header["FILTER"])
    dff = get_dark(ff[0].header["EXPTIME"])
    
    reduced = (im[0].data - df[0].data ) / \
        (ff[0].data - dff[0].data)
    datestr=datetime.utcfromtimestamp\
        (im[0].header["CTIME"]).strftime\
            ("%Y%m%d%H%M%S")
    newname= "%s-df.fits" % (datestr)
    newfile = fits.open(newname, mode='append')
    hdu=fits.PrimaryHDU(data=reduced)
    newfile.append(hdu)
    newfile[0].header = im[0].header
    newfile.close()